In [128]:
import pandas as pd
import os
import re

In [129]:
DATA_PATH = '../data'
PRICES_PATH = os.path.join(DATA_PATH, 'prices')
SIMULATION_DATA_PATH = os.path.join(DATA_PATH, 'simulation data')
prices = pd.read_csv(os.path.join(PRICES_PATH, 'Spain_prices.csv'))

In [130]:
prices['price'] = prices['Price (EUR/MWhe)']
prices['time'] = pd.to_datetime(prices['Datetime (UTC)'])
prices.head()

,Country,ISO3 Code,Datetime (UTC),Datetime (Local),Price (EUR/MWhe),price,time
0,Spain,ESP,2015-01-01 00:00:00,2015-01-01 01:00:00,48.10,48.10,2015-01-01 00:00:00
1,Spain,ESP,2015-01-01 01:00:00,2015-01-01 02:00:00,47.33,47.33,2015-01-01 01:00:00
2,Spain,ESP,2015-01-01 02:00:00,2015-01-01 03:00:00,42.27,42.27,2015-01-01 02:00:00
3,Spain,ESP,2015-01-01 03:00:00,2015-01-01 04:00:00,38.41,38.41,2015-01-01 03:00:00
4,Spain,ESP,2015-01-01 04:00:00,2015-01-01 05:00:00,35.72,35.72,2015-01-01 04:00:00


In [131]:
df_list = []
for file_name in os.listdir(SIMULATION_DATA_PATH):
    df = pd.read_csv(os.path.join(SIMULATION_DATA_PATH, file_name),sep = ',', comment = '#')
    file_name_split = file_name.split('_')
    df['lat_lon'] = file_name_split[2]+ '_' + file_name_split[3]
    df_list.append(df)
production = pd.concat(df_list)

In [132]:
production['time'] = pd.to_datetime(production['time'])
production.head()

,time,local_time,electricity,lat_lon
0,2019-01-01 00:00:00,2019-01-01 01:00,0.251,37.3467_-5.6815
1,2019-01-01 01:00:00,2019-01-01 02:00,0.259,37.3467_-5.6815
2,2019-01-01 02:00:00,2019-01-01 03:00,0.261,37.3467_-5.6815
3,2019-01-01 03:00:00,2019-01-01 04:00,0.248,37.3467_-5.6815
4,2019-01-01 04:00:00,2019-01-01 05:00,0.243,37.3467_-5.6815


In [133]:
production['hour'] = production['time'].dt.hour
production['day'] = production['time'].dt.day
production['month'] = production['time'].dt.month
production['year'] = production['time'].dt.year

prices['hour'] = prices['time'].dt.hour
prices['day'] = prices['time'].dt.day
prices['month'] = prices['time'].dt.month
prices['year'] = prices['time'].dt.year

In [134]:
final_time_price = prices['time'].max() #- pd.Timedelta(days=1) + pd.Timedelta(hours=1)
start_time_price = final_time_price - pd.Timedelta(days=365)

In [135]:
mask_dates = (prices['time'] <= final_time_price) & (prices['time'] > start_time_price)
last_y_prices = prices[mask_dates]

In [136]:
last_y_prices.head()

,Country,ISO3 Code,Datetime (UTC),Datetime (Local),Price (EUR/MWhe),price,time,hour,day,month,year
83469,Spain,ESP,2024-07-09 21:00:00,2024-07-09 23:00:00,116.38,116.38,2024-07-09 21:00:00,21,9,7,2024
83470,Spain,ESP,2024-07-09 22:00:00,2024-07-10 00:00:00,107.94,107.94,2024-07-09 22:00:00,22,9,7,2024
83471,Spain,ESP,2024-07-09 23:00:00,2024-07-10 01:00:00,106.81,106.81,2024-07-09 23:00:00,23,9,7,2024
83472,Spain,ESP,2024-07-10 00:00:00,2024-07-10 02:00:00,104.22,104.22,2024-07-10 00:00:00,0,10,7,2024
83473,Spain,ESP,2024-07-10 01:00:00,2024-07-10 03:00:00,102.61,102.61,2024-07-10 01:00:00,1,10,7,2024


In [137]:
production.head()

,time,local_time,electricity,lat_lon,hour,day,month,year
0,2019-01-01 00:00:00,2019-01-01 01:00,0.251,37.3467_-5.6815,0,1,1,2019
1,2019-01-01 01:00:00,2019-01-01 02:00,0.259,37.3467_-5.6815,1,1,1,2019
2,2019-01-01 02:00:00,2019-01-01 03:00,0.261,37.3467_-5.6815,2,1,1,2019
3,2019-01-01 03:00:00,2019-01-01 04:00,0.248,37.3467_-5.6815,3,1,1,2019
4,2019-01-01 04:00:00,2019-01-01 05:00,0.243,37.3467_-5.6815,4,1,1,2019


In [138]:
df_revenue = pd.merge(production, last_y_prices, on=['hour','day','month'], how='inner')
df_revenue = df_revenue[['lat_lon','price','electricity','hour','day','month']]

In [139]:
# compute asset revenue
df_revenue['revenue'] = df_revenue['price'] * df_revenue['electricity']

In [ ]:
df_revenue.groupby(['lat_lon','month'])['revenue'].sum() # Todo: hourly
df_revenue.pivot_table()


ValueError: No group keys passed!

In [114]:
mask = (prices['day'] == 31) & (prices['month'] == 3) & (prices['year'] == 2025)
prices[mask]

,Country,ISO3 Code,Datetime (UTC),Datetime (Local),Price (EUR/MWhe),price,time,hour,day,month,year
89804,Spain,ESP,2025-03-31 00:00:00,2025-03-31 02:00:00,3.00,3.00,2025-03-31 00:00:00,0,31,3,2025
89805,Spain,ESP,2025-03-31 01:00:00,2025-03-31 03:00:00,0.65,0.65,2025-03-31 01:00:00,1,31,3,2025
89806,Spain,ESP,2025-03-31 02:00:00,2025-03-31 04:00:00,0.65,0.65,2025-03-31 02:00:00,2,31,3,2025
89807,Spain,ESP,2025-03-31 03:00:00,2025-03-31 05:00:00,1.00,1.00,2025-03-31 03:00:00,3,31,3,2025
89808,Spain,ESP,2025-03-31 04:00:00,2025-03-31 06:00:00,7.44,7.44,2025-03-31 04:00:00,4,31,3,2025
89809,Spain,ESP,2025-03-31 05:00:00,2025-03-31 07:00:00,40.03,40.03,2025-03-31 05:00:00,5,31,3,2025
89810,Spain,ESP,2025-03-31 06:00:00,2025-03-31 08:00:00,41.26,41.26,2025-03-31 06:00:00,6,31,3,2025
89811,Spain,ESP,2025-03-31 07:00:00,2025-03-31 09:00:00,0.15,0.15,2025-03-31 07:00:00,7,31,3,2025
89812,Spain,ESP,2025-03-31 08:00:00,2025-03-31 10:00:00,-0.01,-0.01,2025-03-31 08:00:00,8,31,3,2025
89813,Spain,ESP,2025-03-31 09:00:00,2025-03-31 11:00:00,-0.31,-0.31,2025-03-31 09:00:00,9,31,3,2025


In [115]:
prices.loc[89824:89827,:]

,Country,ISO3 Code,Datetime (UTC),Datetime (Local),Price (EUR/MWhe),price,time,hour,day,month,year
89824,Spain,ESP,2025-03-31 20:00:00,2025-03-31 22:00:00,119.85,119.85,2025-03-31 20:00:00,20,31,3,2025
89825,Spain,ESP,2025-04-01 00:00:00,2025-04-01 02:00:00,70.03,70.03,2025-04-01 00:00:00,0,1,4,2025
89826,Spain,ESP,2025-04-01 01:00:00,2025-04-01 03:00:00,60.48,60.48,2025-04-01 01:00:00,1,1,4,2025
89827,Spain,ESP,2025-04-01 02:00:00,2025-04-01 04:00:00,61.59,61.59,2025-04-01 02:00:00,2,1,4,2025
